# VGG16

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation, Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras import Input, models, layers, optimizers, metrics

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
#                                    vertical_flip=True,
                                   fill_mode='nearest')

In [3]:
train_generator = train_datagen.flow_from_directory(
        'VGG16_dataset/train',
        target_size=(150, 150),
        batch_size=3,
        class_mode='categorical')

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
test_generator = test_datagen.flow_from_directory(
        'VGG16_dataset/test',
        target_size=(150, 150),
        batch_size=3,
        class_mode='categorical')

In [7]:
transfer_model=VGG16(weights='imagenet',include_top=False, input_shape=(150,150,3))
transfer_model.trainable=False

In [10]:
# 우리의 모델을 불러오기
finetune_model = models.Sequential()
finetune_model.add(transfer_model) #transfer model을 불러온다.
finetune_model.add(Flatten()) #flatten 해주고
finetune_model.add(Dense(64))
finetune_model.add(Activation('relu')) #여기서 부터 학습
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(2))
finetune_model.add(Activation('softmax'))
finetune_model.summary()

In [8]:
finetune_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [9]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [5]:
history = finetune_model.fit(
       train_generator,
       epochs=10,
       validation_data=test_generator,
       validation_steps=10, 
       callbacks=[early_stopping_callback])